# Implementing RAG with Pinecone

https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/pinecone/Using_Pinecone_for_embeddings_search.ipynb

## Creating Vector Database in Pinecone

### Loading the data

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import ast
from typing import List, Iterator

# Function to set the wd as the root of the repository
def find_repo_root(repo_name):
    current_dir = os.getcwd()
    while current_dir != '/':
        if os.path.basename(current_dir) == repo_name:
            return current_dir
        current_dir = os.path.dirname(current_dir)
    raise FileNotFoundError(f"Repository root '{repo_name}' not found.")

# Setting the working directory
repo_name = 'orare-model'
repo_root = find_repo_root(repo_name)
os.chdir(repo_root)

In [2]:
# Reading the bible interpreted with embeddings
bible_data = pd.read_csv('bible/data/bible_by_theme_int_embedding.txt', sep='|', encoding='utf-8')

# Tranforming vector string into list
bible_data['interpretacion_vector'] = bible_data['interpretacion_vector'].apply(ast.literal_eval)

# Showing the bible_data object
bible_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     861 non-null    object
 1   pasaje                 861 non-null    object
 2   texto                  861 non-null    object
 3   interpretación         861 non-null    object
 4   temas                  861 non-null    object
 5   área_vida              861 non-null    object
 6   texto_vector           861 non-null    object
 7   interpretacion_vector  861 non-null    object
dtypes: object(8)
memory usage: 53.9+ KB


In [3]:
# Formatting the Bible dataframe
# Transforming index as numeric value
bible_data['id'] = bible_data['id'].str.replace('vec', '').astype(int)

# Craeting column "metadata"
bible_data['metadata'] = bible_data.apply(lambda row: {
    'pasaje': row['pasaje'],
    'texto': row['texto'],
    'interpretacion': row['interpretación'],
    'temas': row['temas'],
    'area_vida': row['área_vida']
}, axis=1)

# Dropping the columns inserted in metadata
bible_data = bible_data.drop(columns=['pasaje','texto','interpretación','temas','área_vida'])

# Reordering columns
columns = ['id','interpretacion_vector','metadata']
bible_data = bible_data[columns]
bible_data.head()

,id,interpretacion_vector,metadata
0,1,"[0.05086807906627655, 0.006340509746223688, 0....","{'pasaje': '1 Corintios 10:12', 'texto': 'Así ..."
1,2,"[0.04511460289359093, -0.013647807762026787, -...","{'pasaje': '1 Corintios 10:13', 'texto': 'No o..."
2,3,"[0.0449271984398365, 0.04290250688791275, -0.0...","{'pasaje': '1 Corintios 10:31', 'texto': 'Si p..."
3,4,"[0.06745994836091995, 0.06209307909011841, -0....","{'pasaje': '1 Corintios 11:9', 'texto': 'Porqu..."
4,5,"[0.030204113572835922, -0.004405524581670761, ...","{'pasaje': '1 Corintios 13:13', 'texto': 'Y ah..."


### Setting the connection to OpenAI API and Pinecone API

In [4]:
import openai
import os
import pinecone
from pinecone import Pinecone

# Pinecone API
pc_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pc_api_key)

# OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

/Users/hiroshi/Documents/ds_projects/orare-model/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Helper function

In [11]:
# Models a simple batch generator that make chunks out of an input DataFrame
class BatchGenerator:
   
    def __init__(self, batch_size: int = 10) -> None:
        self.batch_size = batch_size
    
    # Makes chunks out of an input DataFrame
    def to_batches(self, df: pd.DataFrame) -> Iterator[pd.DataFrame]:
        splits = self.splits_num(df.shape[0])
        if splits <= 1:
            yield df
        else:
            for chunk in np.array_split(df, splits):
                yield chunk

    # Determines how many chunks DataFrame contains
    def splits_num(self, elements: int) -> int:
        return int(np.ceil(elements / self.batch_size))
    
    __call__ = to_batches

df_batcher = BatchGenerator(300)

### Creating an Index

In [48]:
# Create the Index in the Pinecone webpage

### Loading the index

In [62]:
index_name = 'bible-verses-metadata'

# Setting the host of the index from the Pinecone web admin portal
index = pc.Index(index_name=index_name, host='https://bible-verses-metadata-rsup9mo.svc.aped-4627-b74a.pinecone.io')

# Confirm our index was created
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'content': {'vector_count': 861}},
 'total_vector_count': 861}

### Uploading the vectors to the content namespace

In [13]:
# Upsert content vectors in content namespace - this can take a few minutes
print("Uploading vectors to content namespace..")
for batch_df in df_batcher(bible_data):
    vectors = list(zip(batch_df.id.astype(str), batch_df.interpretacion_vector, batch_df.metadata))
    index.upsert(vectors=vectors, namespace='content')

Uploading vectors to content namespace..


In [25]:
# Check index size for each namespace to confirm all of our docs have loaded
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'content': {'vector_count': 861}},
 'total_vector_count': 861}

## Vector Search Query

In [ ]:
# Importing the libraries and setting the clients
import pandas as pd
import openai
import os
# Pinecone's client library for Python
import pinecone
from pinecone import Pinecone

# Pinecone API
pc_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pc_api_key)

# OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

In [60]:
# Setting the embedding model of OpenAI
EMBEDDING_MODEL = "text-embedding-3-small"

# Function to query the vector search
def query_article(query, namespace, top_k=3):

    # Function to get the embeddings of the input of the search
    def get_embedding(text, model="text-embedding-3-small"):
       text = text.replace("\n", " ")
       return client.embeddings.create(input = [text], model=model).data[0].embedding
    
    embedded_query = get_embedding(query,model=EMBEDDING_MODEL)

    # Query namespace passed as parameter using title vector
    query_result = index.query(vector=embedded_query, 
                               top_k=top_k,
                               namespace=namespace,
                               include_metadata=True)

    # Print query results 
    print(f"Oración: {query}")
    print('\n')
    for match in query_result['matches']:
        print(f"id: {match.id}, score: {match.score}")
        print(f"pasaje: {match.metadata['pasaje']}")
        print(f"texto: {match.metadata['texto']}")
        print(f"pasaje: {match.metadata['interpretacion']}")
        print('\n')

    return

In [58]:
prayer = '''Dios. Quiero que en este proceso de búsqueda seas mi fuerza y me acompañes en todo momento. 
Yo se que tienes para mi un plan. Yo se que tu siempre das en la medida correcta y que poco a poco me has mostrado el camino para mi crecimiento.
Hace un tiempo me faltaba confianza en mi mismo. Se que con la experiencia laboral anterior me diste la oportunidad de abrir los ojos y saber quien soy.
Te doy las gracias porque se que cada paso que me muestras es un paso hacia desarrollar mi máximo potencial.
'''
query_output = query_article(query=prayer,namespace='content')

Oración: Dios. Quiero que en este proceso de búsqueda seas mi fuerza y me acompañes en todo momento. 
Yo se que tienes para mi un plan. Yo se que tu siempre das en la medida correcta y que poco a poco me has mostrado el camino para mi crecimiento.
Hace un tiempo me faltaba confianza en mi mismo. Se que con la experiencia laboral anterior me diste la oportunidad de abrir los ojos y saber quien soy.
Te doy las gracias porque se que cada paso que me muestras es un paso hacia desarrollar mi máximo potencial.

id: 624, score: 0.551148355
pasaje: Proverbios 3:5, 6
texto: Fíate de Jehová de todo tu corazón, Y no estribes en tu prudencia. Reconócelo en todos tus caminos, Y él enderezará tus veredas.
pasaje: Este pasaje nos enseña a confiar plenamente en Dios y no depender únicamente de nuestra propia sabiduría o entendimiento. Al reconocer a Dios en todas nuestras acciones y decisiones, Él guiará y corregirá nuestro camino.


id: 388, score: 0.536897242
pasaje: Job 11:18, 19
texto: Y confiarás

In [61]:
prayer = '''Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti.
¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, 
pero entiendo espiritualmente lo que estás haciendo conmigo.
Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, 
entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú.
No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, 
respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.
Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, 
aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, 
que su angel guardián le cuente que estamos todos pendientes de él, 
que no contamos los minutos para tenerlo en nuestros brazos, pero que sea paciente,
que tú tienes varios corazones que limpiar en estos momentos, que su venida está generando santidad en nuestro alrededor.'''

query_output = query_article(query=prayer,namespace='content')

Oración: Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti.
¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, 
pero entiendo espiritualmente lo que estás haciendo conmigo.
Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, 
entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú.
No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, 
respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.
Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, 
aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, 
que su angel guardián le cuente que estamos todos pendientes de él,